# Intro to Prompt Engineering with LLMs

This course is produced by the ubiquitous Andrew Ng & OpenAI Engineer Isa Fulford.

The course starts with a distinction in the functionality between 2 types of LLM:

**Base LLM**: Goos at predicting the next predicted word, so can exhibit autocomplete
behaviour. 

**Instruction Tuned LLM**: A base LLM trained to respond to instructions such as 
questions. This often involves **RLHF** - reinforcement learning with human feedback. 

The LLM output should be:

**H**elpful, **H**onest & **H**armless

## Prompt Engineering

To follow along, you'll need:
* to pip install `openai` - this is in the requirements.txt.
* to generate an api key from the [openai website](https://platform.openai.com/account/api-keys).
